In [ ]:
import os
from openai import OpenAI
import os

os.environ["DASHSCOPE_API_KEY"] = "your_api_key_here"
client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"), 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    model="qwen-vl-max", #qwen-vl-plus
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "这是什么"},
                {"type": "image_url","image_url": {"url": "https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg"}}
            ]
        }
        ]
    )
print(completion.model_dump_json())

In [ ]:
messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "框出图中轮毂的位置"},
                {"type": "image_url","image_url": {"url": "https://easycar.oss-cn-beijing.aliyuncs.com/car_undistorted.jpg"}}
            ]
        }
    ]

completion = client.chat.completions.create(
    model="qwen-vl-max-2024-08-09",
    messages=messages
    )
print(completion.model_dump_json())

In [3]:
#help(completion)

In [ ]:
messages.append({'role': 'assistant', 'content': completion.choices[0].message.content})
messages.append({
            "role": "user",
            "content": [
                {"type": "text", "text": "图中轮毂的位置在哪里"},
                {"type": "image_url","image_url": {"url": "https://easycar.oss-cn-beijing.aliyuncs.com/car_undistorted.jpg"}}
            ]
        })
messages

In [ ]:
completion = client.chat.completions.create(
    model="qwen-vl-plus",
    messages=messages
)     

print(completion.model_dump_json())

In [ ]:
completion

In [ ]:
# 调用Qwen-VL本地的模型
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# 设置 PyTorch 的随机种子为 1234，确保随机操作（如模型初始化、数据打乱等）的结果可复现。
# torch.manual_seed(1234)
# Note: The default behavior now has injection attack prevention off.
# 从指定路径加载Qwen-VL-Chat模型的分词器，trust_remote_code=True 表示允许加载远程代码
# AutoTokenizer.from_pretrained 是 transformers 库中的方法，用于从预训练模型路径加载对应的分词器
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/model/Qwen/Qwen-VL-Chat", trust_remote_code=True)
# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cpu", trust_remote_code=True).eval()

# use cuda device
# 使用 transformers 库的 AutoModelForCausalLM.from_pretrained 方法加载预训练的 Qwen-VL-Chat 模型。
# 参数 "Qwen/Qwen-VL-Chat" 指定了要加载的模型名称，这里是阿里云推出的多模态大模型 Qwen-VL 的聊天版本。
# device_map="cuda" 表示将模型加载到 CUDA 设备（即 GPU）上运行，以利用 GPU 的并行计算能力加速模型推理。
# trust_remote_code=True 表示允许加载远程代码，由于 Qwen-VL 模型有自定义的代码实现，需要该参数来确保正确加载。
# .eval() 方法将模型设置为评估模式，这会关闭一些在训练时使用的特殊层（如 Dropout），适合进行推理任务。
# 模型加载到 CUDA 设备（即 GPU）上运行，以利用 GPU 的并行计算能力加速模型推理。
# bf16=True 表示使用 bf16 精度加载模型，这可以在 GPU 上加速推理，同时减少内存占用。
# use_cache=False 表示不使用缓存，这可以在推理时避免一些内存问题。
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/model/Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True, bf16=True,use_cache=False ).eval()

# Specify hyperparameters for generation (No need to do this if you are using transformers>=4.32.0)
# model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)
# 使用分词器的 from_list_format 方法将多模态输入转换为模型可接受的格式
# 该方法接受一个列表，列表中的每个元素是一个字典，可包含图像和文本信息
# 图像可以是本地路径或 URL，这里注释掉的示例展示了另一个图像 URL
query = tokenizer.from_list_format([
    #{'image': 'https://easycar.oss-cn-beijing.aliyuncs.com/2.jpg'}, # Either a local path or an url
    {'image': 'https://easycar.oss-cn-beijing.aliyuncs.com/car_undistorted.jpg'},
    {'text': '这是什么?'},
])

# 调用模型的 chat 方法进行对话交互
# tokenizer 为之前加载的分词器，query 是多模态输入，history 为对话历史，初始为 None
# 该方法返回模型的回复和更新后的对话历史
response, history = model.chat(tokenizer, query=query, history=None,use_cache=False)
# 打印模型的回复
print(response)
# 示例输出：图中展示了一辆蓝色的特斯拉Model3轿车的尾部，从图中可以观察到车辆的尾部有部分凹陷和划痕，但无法确定是否为车辆被追尾。

# 继续调用模型的 chat 方法，传入新的文本指令和之前的对话历史
# 该方法会基于之前的对话上下文给出新的回复
response, history = model.chat(tokenizer, '框出图中轮毂的位置', history=history,use_cache=False)
# 打印新的回复
print(response)
# 示例输出：<ref>轮毂</ref><box>(154,553),(310,880)</box>

# 调用分词器的 draw_bbox_on_latest_picture 方法，根据模型回复中的框选信息在最新图像上绘制边界框
# 该方法接受模型回复和对话历史作为参数，返回绘制好边界框的图像对象
image = tokenizer.draw_bbox_on_latest_picture(response, history)
# 判断是否成功获取到绘制好边界框的图像
if image:
    # 如果获取到图像，将其保存为 wheel.jpg 文件
    image.save('wheel.jpg')
else:
    # 如果未获取到图像，打印提示信息
    print("no box")


In [ ]:

#response = '<ref>轮毂</ref><box>(100,460),(250,880)</box>'
#response = '<ref>轮毂</ref><box>(1,1),(1000,1000)</box>'

# 调用 tokenizer 对象的 draw_bbox_on_latest_picture 方法，该方法的作用是根据传入的 response 和 history 参数，
# 在最新的图片上绘制边界框（bounding box）。response 通常包含边界框的位置信息，
# history 可能包含对话历史或图片相关的上下文信息。方法执行后会返回绘制好边界框的图片对象，若绘制失败则可能返回 None。
image = tokenizer.draw_bbox_on_latest_picture(response, history)
if image:
    image.save('wheel.jpg')
else:
    print("no box")


In [ ]:

response, history = model.chat(tokenizer, '框出图中凹陷和划痕的位置', history=history,use_cache=False )
print(response)
image = tokenizer.draw_bbox_on_latest_picture(response, history)
if image:
    image.save('car_damage.jpg')
else:
    print("no box")


In [ ]:
# 显示gradio和transformers的版本
import gradio
gradio.__version__
import transformers 
transformers.__version__
import torch
torch.__version__

In [ ]:

# 第1轮对话
query = tokenizer.from_list_format([
    {'image': 'https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg'}, # Either a local path or an url
    {'text': '这是什么?'},
])
response, history = model.chat(tokenizer, query=query, history=None)
print(response)
# 图中是一名女子在沙滩上和狗玩耍，旁边是一只拉布拉多犬，它们处于沙滩上。


In [ ]:

# 第1轮对话
query = tokenizer.from_list_format([
    {'image': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/1.jpg'}, 
    {'text': '这是什么？'},
])
response, history = model.chat(tokenizer, query=query, history=None,use_cache=False)
print(response)


In [ ]:

# 第2轮对话
response, history = model.chat(tokenizer, '框出图中包装有缺陷的位置', history=history,use_cache=False)
print(response)
# <ref>击掌</ref><box>(536,509),(588,602)</box>
image = tokenizer.draw_bbox_on_latest_picture(response, history)
if image:
    image.save('2.jpg')
else:
    print("no box")


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# 读取图像
img = mpimg.imread('2.jpg')
# 显示图像
plt.imshow(img)
# 添加标题
plt.title('2.jpg')
# 隐藏坐标轴
plt.axis('off')
# 显示图像
plt.show()


In [ ]:

# 读取图像
img = mpimg.imread('3.jpg')
# 显示图像
plt.imshow(img)
# 隐藏坐标轴
plt.axis('off')
# 显示图像
plt.show()

# 第1轮对话
query = tokenizer.from_list_format([
    {'image': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/3.jpg'}, 
    {'text': '厦门航空在哪个区？'}, 
])
response, history = model.chat(tokenizer, query=query, history=None,use_cache=False)
print(response)

# 第1轮对话
query = tokenizer.from_list_format([
    {'image': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/3.jpg'}, 
    {'text': 'A区有哪些航空公司？'},
])
response, history = model.chat(tokenizer, query=query, history=None,use_cache=False)
print(response)

# 第2轮对话


In [ ]:

# 读取图像
img = mpimg.imread('4.jpg')
# 显示图像
plt.imshow(img)
# 隐藏坐标轴
plt.axis('off')
# 显示图像
plt.show()

# 第1轮对话
query = tokenizer.from_list_format([
    {'image': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/4.jpg'}, 
    {'text': '机场巴士在第几层？'}, 
])
response, history = model.chat(tokenizer, query=query, history=None,use_cache=False)
print(response)


In [ ]:

# 读取图像
img = mpimg.imread('5.png')
# 显示图像
plt.imshow(img)
# 隐藏坐标轴
plt.axis('off')
# 显示图像
plt.show()

# 第1轮对话
query = tokenizer.from_list_format([
    {'image': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/5.png'}, 
    {'text': '头等舱免费行李额是多少KG？'}, 
])
response, history = model.chat(tokenizer, query=query, history=None,use_cache=False)
print(response)
